In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json, math, time
import regex as re
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
KeyPath = %env CODINGDOJO
with open(KeyPath) as f: 
    login = json.load(f)
# other environment variables
data_file = 'tmdb_results_combined.csv.gz'
data_dir = 'data/'
data_basics = 'title_basics.csv.gz'
years_requested = [2000,2001]
tmdb.API_KEY = login['tmdb-api-key-v3']
# if all years ran do not retrive any results, no need to recompile final merged csv
zero_data = True

In [3]:
def get_movie_with_rating(movie_id):
    try:
        movie = tmdb.Movies(movie_id)
        movie_info = movie.info()
        releases = movie.releases()
        x=0
        for c in releases['countries']:
            if c['iso_3166_1'] == 'US':
                x+=1
                movie_info['certification'] = c['certification']
        if x == 0:
            movie_info['certification']=""            
    except:
        movie_info = {'budget':"",'revenue':"",'certification':""}
    json_req = { movie_id :[{'budget':movie_info['budget'],
                             'revenue':movie_info['revenue'],
                             'certification':movie_info['certification']}]}
    return json_req

In [4]:
%%time
for YEAR in tqdm_notebook(years_requested,desc='YEARS',position=0):
    JSON_FILE = f'{data_dir}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    display(file_exists)
    # If it does not exist: create it
    if file_exists == False:
    # create an empty dict if no file exists.
        file_data={}
    else:
        #will keep file_data open as a dict variable
        #so we do not have to read and write, just truncate and dump after 
        #each iteration of the API call 
        with open(JSON_FILE,'r+') as file:
        # load existing data into a dict.
            file_data = json.load(file)
            file.close()


    #get complete list of movie_ids from the basics file for corresponding year 
    movie_ids = pd.read_csv(data_dir+data_basics)[['tconst','startYear']]
    movie_ids = [*movie_ids['tconst'].loc[movie_ids['startYear']==YEAR]]

    # filter out any ids that are already in file_data
    #possible if there are interuptions when extracting data from API
    movie_ids_to_get = [x for x in movie_ids if x not in [*file_data.keys()]]
    #movie_ids_to_get = [x for x in movie_ids if x in ['tt0848228','tt0332280']]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        #print(movie_id)
        json_req = get_movie_with_rating(movie_id)  #This uses your pre-made function
        # Just rewrite entire file. could run again later to update/add results. 
        with open(JSON_FILE,'w') as file:
            file_data.update(json_req)
            json.dump(file_data,file)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    #Already have all data in file_data, no need to read again. 
    #Each year should be saved as a separate .csv.gz file
    #No need if 0 results
    if len(movie_ids_to_get) > 0:
        final_year_df = pd.DataFrame()
        for x in file_data:
            df = pd.read_json(json.dumps(file_data[x]))
            df['tconst']=x
            final_year_df = pd.concat([final_year_df,df])
        final_year_df.to_csv(f"{data_dir}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
        zero_data = False

        

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

False

Movies from 2000:   0%|          | 0/1408 [00:00<?, ?it/s]

False

Movies from 2001:   0%|          | 0/1526 [00:00<?, ?it/s]

Wall time: 21min 8s


In [5]:
%%time
#will always recreate the entire final data file because csv files really cannot be upserted
# and appending can add duplicated rows

#get a list of all csv.gz individual year files 
#https://stackoverflow.com/questions/2225564/get-a-filtered-list-of-files-in-a-directory

files = [f for f in os.listdir(data_dir) if re.match(r'final_tmdb_data_.*\.csv.gz', f)]

#Save a final merged .csv.gz of all of the tmdb api data
#write mode w+ will create file if not exists. a will append to end
if zero_data == False:
    cnt = 1
    for x in files:
        df= pd.read_csv(data_dir+x)
        if cnt == 1:
            df.to_csv(data_dir+data_file,mode='w+' ,compression="gzip", index=False)
            cnt+=1
        elif cnt > 1:
            df.to_csv(data_dir+data_file,mode='a' ,compression="gzip", index=False,header=False)

Wall time: 327 ms
